# Mapping Te Papa&rsquo;s collections

This notebook creates some simple maps using the `production.spatial` facet of the Te Papa API to identify places where collection objects were created.

<div class="alert alert-block alert-warning">
<p>If you haven't used one of these notebooks before, they're basically web pages in which you can write, edit, and run live code. They're meant to encourage experimentation, so don't feel nervous. Just try running a few cells and see what happens!.</p>

<p>
    Some tips:
    <ul>
        <li>Code cells have boxes around them.</li>
        <li>To run a code cell click on the cell and then hit <b>Shift+Enter</b>. The <b>Shift+Enter</b> combo will also move you to the next cell, so it's a quick way to work through the notebook.</li>
        <li>While a cell is running a <b>*</b> appears in the square brackets next to the cell. Once the cell has finished running the asterix will be replaced with a number.</li>
        <li>In most cases you'll want to start from the top of notebook and work your way down running each cell in turn. Later cells might depend on the results of earlier ones.</li>
        <li>To edit a code cell, just click on it and type stuff. Remember to run the cell once you've finished editing.</li>
    </ul>
</p>
</div>

In [1]:
import requests
import pandas as pd
import altair as alt
import re
import folium
from tqdm.auto import tqdm
from folium.plugins import MarkerCluster
from IPython.display import display, HTML

## Get an API key
[Sign up here](https://data.tepapa.govt.nz/docs/register.html) for your very own API key.

In [ ]:
# Insert your API key between the quotes
api_key = 'YOUR API KEY'
# If you don't have an API key yet, you can leave the above blank and we'll pick up a guest token below
print('Your API key is: {}'.format(api_key))

## Set some parameters

In [3]:
search_endpoint = 'https://data.tepapa.govt.nz/collection/search'

headers = {
    'x-api-key': api_key,
    'Accept': 'application/json'
}

if not api_key:
    response = requests.get('https://data.tepapa.govt.nz/collection/search')
    data = response.json()
    guest_token = data['guestToken']
    headers['Authorization'] = 'Bearer {}'.format(guest_token)

Below we set the search parameters. Currently it will return information about all objects in the collection. You can change the `query` value to limit the result set — try replacing the asterix with some keywords.

The `size` parameter sets the number of places to return — so in this case we're getting the 100 places that have the most objects associated with them.

The `production.spatial.href` facet gives us the API url of the place itself, so we can use it to get more information about the place.

In [4]:
post_data = {
    'query': '*',
    'filters': [{
        'field': 'type',
        'keyword': 'Object'
    }],
    'facets': [
        {'field': 'production.spatial.href',
        'size': 100}
    ]
}

## Get some data

In [5]:
# Make the API request
response = requests.post(search_endpoint, json=post_data, headers=headers)
data = response.json()

In [6]:
# Convert the facets data to a dataframe and do some cleaning up
# We end up with two columns -- one with the place url, and the other with the number of objects associated with that place
places_df = pd.DataFrame(list(data['facets']['production.spatial.href'].items()))
places_df.columns = ['place_id', 'count']
places_df.head()

,place_id,count
0,https://data.tepapa.govt.nz/collection/place/2...,536
1,https://data.tepapa.govt.nz/collection/place/2...,9698
2,https://data.tepapa.govt.nz/collection/place/2...,430
3,https://data.tepapa.govt.nz/collection/place/2...,554
4,https://data.tepapa.govt.nz/collection/place/2...,466


## Add more information about each place

Using the place url we'll get the full record for each place. We'll then save the name of the place, its geospatial coordinates (if any), and its ISO country code (if any) to the dataframe.

In [7]:
tqdm.pandas()

def find_country_code(place):
    code = None
    if 'alternativeTerms' in place:
        for term in place['alternativeTerms']:
            try:
                if term[:3] == 'ISO':
                    code = term[3:]
            except TypeError:
                pass
    return code       

def add_place_info(place_id):
    response = requests.get(place_id, headers=headers)
    place_data = response.json()
    code = find_country_code(place_data)
    if 'geoLocation' in place_data:
        lat = place_data['geoLocation']['lat']
        lon = place_data['geoLocation']['lon']
    else:
        lat = None
        lon = None
    return pd.Series([place_data['title'], lat, lon, code])

places_df[['title', 'lat', 'lon', 'isocode']] = places_df['place_id'].progress_apply(add_place_info)
places_df.head()

  0%|          | 0/100 [00:00<?, ?it/s]

,place_id,count,title,lat,lon,isocode
0,https://data.tepapa.govt.nz/collection/place/2...,536,Surrey (United Kingdom),51.200,-0.050,None
1,https://data.tepapa.govt.nz/collection/place/2...,9698,Auckland (New Zealand),-36.917,174.783,None
2,https://data.tepapa.govt.nz/collection/place/2...,430,Kanto (Nihon),36.250,139.500,None
3,https://data.tepapa.govt.nz/collection/place/2...,554,D'Urville Island (New Zealand),NaN,NaN,None
4,https://data.tepapa.govt.nz/collection/place/2...,466,Solomon Islands,NaN,NaN,None


## Make a map

In [8]:
import html
m = folium.Map(
    location=[10, 10],
    zoom_start=1.5
)
# We'll cluster the markers for better readability
marker_cluster = MarkerCluster().add_to(m)

for index, row in places_df.dropna(subset=['lat', 'lon']).iterrows():
    # We can easily change the API url to a web url and use it to link the map to the Te Papa collection web site
    web_url = row['place_id'].replace('/collection/', '/').replace('data', 'collections')
    popup = '<b><a target="_blank" href="{}">{}</a></b><br>{} objects'.format(web_url, html.escape(row['title']), row['count'])
    folium.Marker([row['lat'], row['lon']], popup=popup).add_to(marker_cluster)
    
m

## Make another map

Let's try and make the **number** of objects created in each place more obvious.

In [9]:
import html
m = folium.Map(
    location=[10, 10],
    zoom_start=1.5
)

for index, row in places_df.dropna(subset=['lat', 'lon']).iterrows():
    popup = '<b>{}</b><br>{} objects'.format(html.escape(row['title']), row['count'])
    folium.Circle([row['lat'], row['lon']], radius=row['count']*5, popup=popup, color='#de2d26', fill=True).add_to(m)
    
m

## What's missing?

Remember that we're not seeing **all** the places where objects were created. First of all the facet `size` parameter limited out results to the top 100 places. Trying changing it to see what happens.

Even amongst the top 100, not every place had geospatial coordinates attached to it. So not everything is on the map. Let's create a list of places without coordinates.

In [10]:
places_df.loc[places_df['lat'].isnull()]  

,place_id,count,title,lat,lon,isocode
3,https://data.tepapa.govt.nz/collection/place/2...,554,D'Urville Island (New Zealand),NaN,NaN,None
4,https://data.tepapa.govt.nz/collection/place/2...,466,Solomon Islands,NaN,NaN,None
34,https://data.tepapa.govt.nz/collection/place/2...,682,Chatham Islands (New Zealand),NaN,NaN,None
45,https://data.tepapa.govt.nz/collection/place/2...,435,Jawa (Indonesia),NaN,NaN,None
46,https://data.tepapa.govt.nz/collection/place/2...,5228,North Island (New Zealand),NaN,NaN,None
58,https://data.tepapa.govt.nz/collection/place/2...,769,South Island (New Zealand),NaN,NaN,None
64,https://data.tepapa.govt.nz/collection/place/2...,409,Africa,NaN,NaN,None
70,https://data.tepapa.govt.nz/collection/place/2...,1147,Stewart Island (New Zealand),NaN,NaN,None
71,https://data.tepapa.govt.nz/collection/place/2...,285,Pacific Islands,NaN,NaN,None
77,https://data.tepapa.govt.nz/collection/place/2...,433,Czechoslovakia,NaN,NaN,None


----

Created by [Tim Sherratt](https://timsherratt.org/) for the [GLAM Workbench](https://glam-workbench.net/). Support this project by becoming a [GitHub sponsor](https://github.com/sponsors/wragge?o=esb).